# Cleaning

In [1]:
import chardet
import pandas as pd
import datetime

In [2]:
#with open('data/owner.csv', 'rb') as f:
#    result = chardet.detect(f.read())  # or readline if the file is large
#
#print(result)

In [3]:
# display full url for user_profile for instance
#pd.set_option('display.max_colwidth', 1000)

In [4]:
owner = pd.read_csv("data/owner.csv",sep=';', encoding='ISO-8859-1')
prop = pd.read_csv("data/property.csv", sep=';', encoding='ISO-8859-1')

In [5]:
print("shape of owner is :" + str(owner.shape))
print("shape of prop is :" + str(prop.shape))

shape of owner is :(15127, 7)
shape of prop is :(18686, 14)


In [6]:
# vérification des années
#owner["Année de Birthdate"].unique()

In [7]:
# make a dictionary that maps the month name in french to a number
french_to_eng = {'janvier':'1','février':'2','mars': '3','avril': '4','mai':'5','juin':'6','juillet': '7','août': '8','septembre': '9','octobre':'10','novembre':'11','décembre':'12'}

# make new columsn for day month and year. For month, map the french name to month numbers
owner['day_sign_up'] = owner['Jour de Created At User'].apply(lambda x : x.split(' ')[0])
owner['month_sign_up'] = owner['Jour de Created At User'].apply(lambda x : x.split(' ')[1]).map(french_to_eng)
owner['year_sign_up'] = owner['Jour de Created At User'].apply(lambda x : x.split(' ')[2])

# make date time column from year, month and day.
owner['user_sign_up_date'] = pd.to_datetime(owner['year_sign_up']+'-'+owner['month_sign_up']+'-'+owner['day_sign_up'],format='%Y-%m-%d', errors='ignore')

owner = owner.drop(columns=['day_sign_up', 'month_sign_up', 'year_sign_up'])

In [8]:
owner['Jour de Last Login Date'].fillna(owner['Jour de Created At User'], inplace=True)

In [9]:
# make new columsn for day month and year. For month, map the french name to month numbers
owner['day_sign_in'] = owner['Jour de Last Login Date'].apply(lambda x : x.split(' ')[0])
owner['month_sign_in'] = owner['Jour de Last Login Date'].apply(lambda x : x.split(' ')[1]).map(french_to_eng)
owner['year_sign_in'] = owner['Jour de Last Login Date'].apply(lambda x : x.split(' ')[2])

# make date time column from year, month and day.
owner['user_sign_in_date'] = pd.to_datetime(owner['year_sign_in']+'-'+owner['month_sign_in']+'-'+owner['day_sign_in'],format='%Y-%m-%d', errors='ignore')

owner = owner.drop(columns=['day_sign_in', 'month_sign_in', 'year_sign_in'])

In [10]:
owner['user_sign_in_date'] = pd.to_datetime(owner['user_sign_in_date'])
owner['user_sign_up_date'] = pd.to_datetime(owner['user_sign_up_date'])

In [11]:
owner['today_date']= datetime.date.today()
owner['today_date']= pd.to_datetime(owner.today_date)

In [12]:
owner['seniority'] = owner.today_date - owner.user_sign_in_date
owner['days_since_last_login'] = owner.today_date - owner.user_sign_up_date

# quand owner['days_since_last_login'] vaut 0, on émet l'hypothèse que le compte est inactif

In [13]:
owner['year'] = datetime.date.today().year
owner['Année de Birthdate'] = owner['Année de Birthdate'].fillna(datetime.date.today().year)
owner['age'] = owner.year - owner['Année de Birthdate']
owner = owner.drop(columns=['Année de Birthdate', 'year', 'Jour de Created At User', 'Jour de Last Login Date'])

In [15]:
# Je ne traite pas les 28 lignes présentant des erreurs de formatages
df_owner = owner[owner["Languages"].str.startswith('{') == False]
df_owner = df_owner.rename(columns={"owner_id":"id_owner","gender":"gaaender","languages":"languages","country":"country","user_sign_up_date":"user_sign_up_date","user_sign_in_date":"user_sign_in_date","today_date":"today_date","seniority":"seniority","days_since_last_login":"days_since_last_login","age": "age"})
df_owner.columns = map(str.lower, df_owner.columns)
df_owner.columns = df_owner.columns.str.replace(' ', '_')
print(df_owner.shape)
print(owner.shape)
print(owner[owner["Languages"].str.startswith('{')].shape)

(15099, 10)
(15127, 10)
(28, 10)


In [14]:
# On supprime les ages avec des valeurs négatives
df_owner['age'][df_owner.age <= -1] = 0

,Owner ID,Gender,Languages,Country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
0,0a0e4-ad50-4b3a-81fd-a29ed0000,male,[],NaN,2018-05-12,2019-02-19,2019-06-17,118 days,401 days,0.0
1,0a1e1-1cfe-46d0-9069-e567bc9c3,female,[],NaN,2018-06-11,2018-06-26,2019-06-17,356 days,371 days,0.0
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,female,[],NaN,2018-04-17,2018-04-17,2019-06-17,426 days,426 days,0.0
3,0a2bd-8842-4067-b52e-0a8922be2,female,"[""en"", ""fr""]",NaN,2017-06-26,2017-07-17,2019-06-17,700 days,721 days,0.0
4,0a3ef-f75e-463c-b52b-fa39184d5,male,[],NaN,2018-03-09,2018-03-09,2019-06-17,465 days,465 days,0.0


In [16]:
# Boolean values
funrished_mapping= {'FAUX':False, 'VRAI':True}
prop.Furnished = prop.Furnished.map(funrished_mapping)
prop.Furnished.unique()

array([nan, False, True], dtype=object)

In [17]:
# make a dictionary that maps the month name in french to a number
french_to_eng = {'janv':'1','févr':'2','mars': '3','avr': '4','mai':'5','juin':'6','juil': '7','août': '8','sept': '9','oct':'10','nov':'11','déc':'12'}
year_mapping = {'13':'2013','14':'2014','15':'2015','16':'2016','17':'2017','18':'2018', '19':'2019'}

# make new columsn for day month and year. For month, map the french name to month numbers
prop['day_offline_at'] = prop[prop['Jour de Offline At'].notnull() == True]['Jour de Offline At'].apply(lambda x : x.split('-')[0])
prop['month_offline_at'] = prop[prop['Jour de Offline At'].notnull() == True]['Jour de Offline At'].apply(lambda x : x.split('-')[1]).map(french_to_eng)
prop['year_offline_at'] = prop[prop['Jour de Offline At'].notnull() == True]['Jour de Offline At'].apply(lambda x : x.split('-')[2]).map(year_mapping)

# make date time column from year, month and day.
prop['offline_date'] = pd.to_datetime(prop['year_offline_at']+'-'+prop['month_offline_at']+'-'+prop['day_offline_at'],format='%Y-%m-%d', errors='ignore')

prop = prop.drop(columns=['day_offline_at', 'month_offline_at', 'year_offline_at'])

In [18]:
# make a dictionary that maps the month name in french to a number

# make new columsn for day month and year. For month, map the french name to month numbers
prop['day_online_at'] = prop[prop['Jour de Online At'].notnull() == True]['Jour de Online At'].apply(lambda x : x.split('-')[0])
prop['month_online_at'] = prop[prop['Jour de Online At'].notnull() == True]['Jour de Online At'].apply(lambda x : x.split('-')[1]).map(french_to_eng)
prop['year_online_at'] = prop[prop['Jour de Online At'].notnull() == True]['Jour de Online At'].apply(lambda x : x.split('-')[2]).map(year_mapping)

# make date time column from year, month and day.
prop['online_date'] = pd.to_datetime(prop['year_online_at']+'-'+prop['month_online_at']+'-'+prop['day_online_at'],format='%Y-%m-%d', errors='ignore')

prop = prop.drop(columns=['day_online_at', 'month_online_at', 'year_online_at'])

In [19]:
# make a dictionary that maps the month name in french to a number

# make new columsn for day month and year. For month, map the french name to month numbers
prop['day_created_at'] = prop[prop['Jour de Created At (Properties)'].notnull() == True]['Jour de Created At (Properties)'].apply(lambda x : x.split('-')[0])
prop['month_created_at'] = prop[prop['Jour de Created At (Properties)'].notnull() == True]['Jour de Created At (Properties)'].apply(lambda x : x.split('-')[1]).map(french_to_eng)
prop['year_created_at'] = prop[prop['Jour de Created At (Properties)'].notnull() == True]['Jour de Created At (Properties)'].apply(lambda x : x.split('-')[2]).map(year_mapping)

# make date time column from year, month and day.
prop['proprety_creation_date'] = pd.to_datetime(prop['year_created_at']+'-'+prop['month_created_at']+'-'+prop['day_created_at'],format='%Y-%m-%d', errors='ignore')

prop = prop.drop(columns=['day_created_at', 'month_created_at', 'year_created_at','Jour de Offline At','Jour de Online At','Jour de Created At (Properties)'])

In [20]:
# Delete first row
df_prop = prop[1:-1]

In [21]:
df_prop.columns = map(str.lower, df_prop.columns)
df_prop.columns = df_prop.columns.str.rstrip()
df_prop.columns = df_prop.columns.str.replace(' ', '_')

In [22]:
df_owner.head()

,owner_id,gender,languages,country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
0,0a0e4-ad50-4b3a-81fd-a29ed0000,male,[],NaN,2018-05-12,2019-02-19,2019-06-17,118 days,401 days,0.0
1,0a1e1-1cfe-46d0-9069-e567bc9c3,female,[],NaN,2018-06-11,2018-06-26,2019-06-17,356 days,371 days,0.0
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,female,[],NaN,2018-04-17,2018-04-17,2019-06-17,426 days,426 days,0.0
3,0a2bd-8842-4067-b52e-0a8922be2,female,"[""en"", ""fr""]",NaN,2017-06-26,2017-07-17,2019-06-17,700 days,721 days,0.0
4,0a3ef-f75e-463c-b52b-fa39184d5,male,[],NaN,2018-03-09,2018-03-09,2019-06-17,465 days,465 days,0.0


# Analysis

In [41]:
import numpy as np

In [29]:
df_owner.groupby('gender').count().owner_id

gender
female    7495
male      7604
Name: owner_id, dtype: int64

In [24]:
import plotly.plotly as py
import plotly.graph_objs as go

In [25]:
trace = go.Scatter(
    x = df_owner[['owner_id','days_since_last_login']].owner_id.tolist(),
    y = df_owner['days_since_last_login'].apply(lambda x : x.days).tolist(),
    mode = 'markers'
)

data = [trace]
py.iplot(data, filename='basic-scatter')

/Users/noesautel/miniconda3/envs/develop/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



On distingue deux groupes d'utilisateurs: les actifs ayant réaliser une connexion au cours des 500 derniers jours, et ceux qui semblent avoir quitter la plateforme il y a envion 700 jours.

In [34]:
df_owner.dtypes

owner_id                          object
gender                            object
languages                         object
country                           object
user_sign_up_date         datetime64[ns]
user_sign_in_date         datetime64[ns]
today_date                datetime64[ns]
seniority                timedelta64[ns]
days_since_last_login    timedelta64[ns]
age                              float64
dtype: object

In [35]:
df_owner.head()

,owner_id,gender,languages,country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
0,0a0e4-ad50-4b3a-81fd-a29ed0000,male,[],NaN,2018-05-12,2019-02-19,2019-06-17,118 days,401 days,0.0
1,0a1e1-1cfe-46d0-9069-e567bc9c3,female,[],NaN,2018-06-11,2018-06-26,2019-06-17,356 days,371 days,0.0
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,female,[],NaN,2018-04-17,2018-04-17,2019-06-17,426 days,426 days,0.0
3,0a2bd-8842-4067-b52e-0a8922be2,female,"[""en"", ""fr""]",NaN,2017-06-26,2017-07-17,2019-06-17,700 days,721 days,0.0
4,0a3ef-f75e-463c-b52b-fa39184d5,male,[],NaN,2018-03-09,2018-03-09,2019-06-17,465 days,465 days,0.0


In [54]:
df_test = df_owner

In [55]:
df_test.head(4)

,owner_id,gender,languages,country,user_sign_up_date,user_sign_in_date,today_date,seniority,days_since_last_login,age
0,0a0e4-ad50-4b3a-81fd-a29ed0000,male,[],NaN,2018-05-12,2019-02-19,2019-06-17,118 days,401 days,0.0
1,0a1e1-1cfe-46d0-9069-e567bc9c3,female,[],NaN,2018-06-11,2018-06-26,2019-06-17,356 days,371 days,0.0
2,0a2b8-f8c2-4a2c-b6f8-d2f901abe,female,[],NaN,2018-04-17,2018-04-17,2019-06-17,426 days,426 days,0.0
3,0a2bd-8842-4067-b52e-0a8922be2,female,"[""en"", ""fr""]",NaN,2017-06-26,2017-07-17,2019-06-17,700 days,721 days,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15099 entries, 0 to 15126
Data columns (total 10 columns):
owner_id                 15099 non-null object
gender                   15099 non-null object
languages                15099 non-null object
country                  1540 non-null object
user_sign_up_date        15099 non-null datetime64[ns]
user_sign_in_date        15099 non-null datetime64[ns]
today_date               15099 non-null datetime64[ns]
seniority                15099 non-null int64
days_since_last_login    15099 non-null timedelta64[ns]
age                      15099 non-null float64
dtypes: datetime64[ns](3), float64(1), int64(1), object(4), timedelta64[ns](1)
memory usage: 1.3+ MB


In [56]:
# transformation de datetime64 vers int64

df_owner.seniority = df_owner.seniority.apply(lambda x: x/np.timedelta64(1, 'D')).astype(int)
df_owner.days_since_last_login = df_owner.days_since_last_login.apply(lambda x: x/np.timedelta64(1, 'D')).astype(int)

In [59]:
df_owner.days_since_last_login = df_owner.days_since_last_login.apply(lambda x: x/np.timedelta64(1, 'D')).astype(int)

In [63]:
df_prop.head(2)

,id_property,id_owner,furnished,max_tenant_number,property_type,status,bedrooms_count,ville,property_surface,rooms_count,rent,offline_date,online_date,proprety_creation_date
1,0a0cd-0698-434d-a04a-339904547,392d0-46db-4c8b-80f4-c106b5ccf,False,1.0,apartment,offline,NaN,Paris-Ivry-Villejuif,30,1.0,NaN,2016-10-02,2018-03-03,2016-11-17
2,0a1b8-576c-4a89-a8db-32ad0c041,48b60-51f7-4adf-8a92-1ee106c88,True,2.0,apartment,offline,1.0,Toulouse,45,2.0,950,2018-07-15,2018-05-24,2018-05-23


In [71]:
df_prop.max_tenant_number = df_prop.max_tenant_number.astype(int)

/Users/noesautel/miniconda3/envs/develop/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [103]:
df_test = df_prop

In [191]:
df_test.property_surface.unique()

array(['30', '45', '10', '35', '28', '260', '200', '21', '120', '16',
       '24', '25', '18', '42', '60', '104', '70', '164', '55', '575',
       '288', '175', '17', '23', '26', '27', '15', '80', '50', '39', '72',
       '243', '29', '32', '118', '20', '9', '33', '222', '280', '54',
       '300', '36', '46', '130', '106', '465', '132', '460', '13', '34',
       '37', '344', '270', '231', '219', '52', '540', '210', '19', '392',
       '100', '126', '304', '110', '65', '31', '150', '144', '22', '14',
       '160', '197', '180', '234', '40', '62', '265', '12', '360', '11',
       '228', '82', '75', '90', '48', '340', '92', '84', '320', '240',
       '41', '122', '380', '85', '47', '710', '112', '276', '78', '249',
       '255', '108', '230', '64', '279', '74', '95', '102', '66', '86',
       '61', '450', '140', '38', '94', '600', '67', '105', '400', '71',
       '490', '57', '170', '182', '49', '156', '44', '264', '207', '58',
       '720', '960', '480', '171', '520', '201', '404', '128'

In [190]:
# ['Jour de Offline At'].apply(lambda x : x.split('-')[0])
# owner[owner["Languages"].str.startswith('{') == False]
df_test[df_test.property_surface.str.isdigit() == False].property_surface.str[2:].astype(int)+1000


640      1260
709      1435
922      1190
998      1250
1707     1600
1767     1200
2147     1050
2303     1200
2316     1470
2720     1200
2772     1080
3008     1071
3273     1036
3390     1260
3465     1200
3670     1800
3698     1015
3772     1000
4083     1400
4565     1000
4568     1520
4912     1080
4939     1800
5168     1400
5841     1000
5972     1190
6167     1400
6459     1940
7183     1000
7715     1200
         ... 
11414    1000
11454    1400
11727    1701
11973    1120
12380    1344
12385    1540
12693    1380
12929    1120
13168    1080
13213    1600
13214    1000
13311    1120
13316    1680
13586    1350
13832    1350
14024    1080
14585    1080
14614    1900
14770    1400
15133    1400
15957    1300
16152    1400
16296    1162
16356    1400
16502    1120
16763    1260
17042    1250
17613    1040
17778    1000
18299    1190
Name: property_surface, Length: 82, dtype: int64

In [210]:
df_test[(df_test.property_surface.str.isdigit() == False) & (df_test.property_surface.str.startswith('2') == True)] = df_test[(df_test.property_surface.str.isdigit() == False) & (df_test.property_surface.str.startswith('2') == True)].property_surface.str[2:].astype(int)+2000

/Users/noesautel/miniconda3/envs/develop/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



ValueError: Must have equal len keys and value when setting with an iterable

In [208]:
df_test.property_surface.astype(str).unique()

array(['30', '45', '10', '35', '28', '260', '200', '21', '120', '16',
       '24', '25', '18', '42', '60', '104', '70', '164', '55', '575',
       '288', '175', '17', '23', '26', '27', '15', '80', '50', '39', '72',
       '243', '29', '32', '118', '20', '9', '33', '222', '280', '54',
       '300', '36', '46', '130', '106', '465', '132', '460', '13', '34',
       '37', '344', '270', '231', '219', '52', '540', '210', '19', '392',
       '100', '126', '304', '110', '65', '31', '150', '144', '22', '14',
       '160', '197', '180', '234', '40', '62', '265', '12', '360', '11',
       '228', '82', '75', '90', '48', '340', '92', '84', '320', '240',
       '41', '122', '380', '85', '47', '710', '112', '276', '78', '249',
       '255', '108', '230', '64', '279', '74', '95', '102', '66', '86',
       '61', '450', '140', '38', '94', '600', '67', '105', '400', '71',
       '490', '57', '170', '182', '49', '156', '44', '264', '207', '58',
       '720', '960', '480', '171', '520', '201', '404', '128'

In [101]:

df_test.head(1)

1    NaN
Name: property_surface, dtype: object

In [99]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18684 entries, 1 to 18684
Data columns (total 14 columns):
id_property               18684 non-null object
id_owner                  18684 non-null object
furnished                 18684 non-null object
max_tenant_number         18684 non-null int64
property_type             18684 non-null object
status                    18684 non-null object
bedrooms_count            17305 non-null float64
ville                     18424 non-null object
property_surface          18684 non-null object
rooms_count               18684 non-null float64
rent                      17305 non-null object
offline_date              15455 non-null datetime64[ns]
online_date               17495 non-null datetime64[ns]
proprety_creation_date    18684 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(2), int64(1), object(8)
memory usage: 2.0+ MB


In [147]:
code={'\\xa':" "}

#df_test.property_surface = 
df_test.property_surface.split()

1         30
2         45
3         10
4         35
5         35
6         28
7        260
8        200
9         21
10       120
11        16
12        24
13        28
14        25
15        18
16        42
17        60
18       104
19        42
20        70
21        35
22       164
23        55
24        18
25       575
26       288
27       175
28       120
29        17
30        23
        ... 
18655     25
18656     24
18657     31
18658     20
18659     18
18660     25
18661     17
18662     25
18663     23
18664     22
18665     23
18666     36
18667    150
18668     80
18669    168
18670    201
18671     28
18672     17
18673    380
18674     18
18675     22
18676     28
18677     20
18678     26
18679     33
18680     23
18681     23
18682     10
18683     90
18684     42
Name: property_surface, Length: 18684, dtype: object

In [80]:
df_prop[df_prop.property_surface == '1\xa0260']

,id_property,id_owner,furnished,max_tenant_number,property_type,status,bedrooms_count,ville,property_surface,rooms_count,rent,offline_date,online_date,proprety_creation_date
640,1d13b-76be-414e-baf9-b24ed4587,e9d5d-b763-460f-b2d3-a5c572109,True,9,house,online,81.0,Lille,1 260,108.0,30 132,2018-08-16,2019-06-04,2017-07-26
3390,7ecfb-39e3-47a0-a7cf-37589ce24,fd7ef-1532-40d8-acff-843457b86,True,7,house,offline,49.0,Le Havre,1 260,49.0,19 600,2018-09-30,2018-07-16,2018-07-10
10962,64764-c2e2-4a12-be50-16b87a56c,44747-1c42-4651-8ce7-121752c4a,True,7,house,online,49.0,Bordeaux,1 260,56.0,25 305,2019-03-27,2019-05-20,2019-03-05
16763,e81fd-60e8-4fdc-91d7-885eded7b,8a55f-a1a8-4895-b72e-7c8265d78,True,2,house,online,36.0,Cergy,1 260,48.0,"2 100,000000000",NaT,2019-03-29,2019-02-09


In [78]:
df_prop.property_surface.unique()

array(['30', '45', '10', '35', '28', '260', '200', '21', '120', '16',
       '24', '25', '18', '42', '60', '104', '70', '164', '55', '575',
       '288', '175', '17', '23', '26', '27', '15', '80', '50', '39', '72',
       '243', '29', '32', '118', '20', '9', '33', '222', '280', '54',
       '300', '36', '46', '130', '106', '465', '132', '460', '13', '34',
       '37', '344', '270', '231', '219', '52', '540', '210', '19', '392',
       '100', '126', '304', '110', '65', '31', '150', '144', '22', '14',
       '160', '197', '180', '234', '40', '62', '265', '12', '360', '11',
       '228', '82', '75', '90', '48', '340', '92', '84', '320', '240',
       '41', '122', '380', '85', '47', '710', '112', '276', '78', '249',
       '255', '108', '230', '64', '279', '74', '95', '102', '66', '86',
       '61', '450', '140', '38', '94', '600', '67', '105', '400', '71',
       '490', '57', '170', '182', '49', '156', '44', '264', '207', '58',
       '720', '960', '480', '171', '520', '201', '404', '128'

In [72]:
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18684 entries, 1 to 18684
Data columns (total 14 columns):
id_property               18684 non-null object
id_owner                  18684 non-null object
furnished                 18684 non-null object
max_tenant_number         18684 non-null int64
property_type             18684 non-null object
status                    18684 non-null object
bedrooms_count            17305 non-null float64
ville                     18424 non-null object
property_surface          18684 non-null object
rooms_count               18684 non-null float64
rent                      17305 non-null object
offline_date              15455 non-null datetime64[ns]
online_date               17495 non-null datetime64[ns]
proprety_creation_date    18684 non-null datetime64[ns]
dtypes: datetime64[ns](3), float64(2), int64(1), object(8)
memory usage: 2.0+ MB


In [28]:
-> changer la data des proprety surface

SyntaxError: invalid syntax (<ipython-input-28-193f304fa3db>, line 1)